In [5]:
from alhazen import Alhazen

from alhazen_formalizations.calculator import initial_inputs, grammar_alhazen as grammar, prop
from alhazen.helper import show_tree

In [6]:
MAX_ITERATION = 30
alhazen = Alhazen(
    initial_inputs=initial_inputs,
    grammar=grammar,
    evaluation_function=prop,
    max_iter=MAX_ITERATION
)
trees = alhazen.run()

In [46]:
from alhazen.feature_collector import Collector
from sklearn.tree import export_text
import numpy as np

from alhazen.oracle import OracleResult


collector = Collector(grammar=grammar)

tree = trees[MAX_ITERATION-1].tree_

paths = []

def search_path(tree, id, path_ids):
    
    path_ids.append(id)

    if tree.children_left[id] != tree.children_right[id]:  
        search_path(tree, tree.children_left[id], path_ids)
        search_path(tree, tree.children_right[id], path_ids)
    else:
        value = tree.value[id]
        class_index = np.argmax(value)
        class_name = trees[MAX_ITERATION-1].classes_[class_index]
        if class_index == 0:
            return OracleResult.BUG, path_ids

    
print(search_path(tree, 0, []))











None


In [48]:
clf = trees[MAX_ITERATION-1]


n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold
impurity = clf.tree_.impurity
value = clf.tree_.value


def retrieve_branches(number_nodes, children_left_list, children_right_list):
    """Retrieve decision tree branches"""
    
    # Calculate if a node is a leaf
    is_leaves_list = [(False if cl != cr else True) for cl, cr in zip(children_left_list, children_right_list)]
    
    # Store the branches paths
    paths = []
    
    for i in range(number_nodes):
        if is_leaves_list[i]:
            # Search leaf node in previous paths
            end_node = [path[-1] for path in paths]

            # If it is a leave node yield the path
            if i in end_node:
                output = paths.pop(np.argwhere(i == np.array(end_node))[0][0])
                yield output

        else:
            
            # Origin and end nodes
            origin, end_l, end_r = i, children_left_list[i], children_right_list[i]

            # Iterate over previous paths to add nodes
            for index, path in enumerate(paths):
                if origin == path[-1]:
                    paths[index] = path + [end_l]
                    paths.append(path + [end_r])

            # Initialize path in first iteration
            if i == 0:
                paths.append([i, children_left[i]])
                paths.append([i, children_right[i]])

all_branches = list(retrieve_branches(n_nodes, children_left, children_right))
print(all_branches)

[[0, 1], [0, 2, 3], [0, 2, 4]]
